In [13]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV
from splitrepeat import splitrepeat_cv

config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

features = ['ADAS_Q1', 'ADAS_Q8', 'ADAS_Q4', 'LDELTOTAL', 'AVDEL30MIN', 'faq9', 'tmab_time', 'CATANIMSC', 'faq2', 'faq3', 'faq10', 'e_memory_pt', 'e_plan_pt', 'e_memory_cg', 'e_lang_cg', 'e_plan_cg', 'e_divatt_cg']

categorical_features = np.in1d(features, ['MMONFLR', 'faq10', 'faq6', 'COPYSYM', 'COPYNUM', 'faq5', 'MMTREE', 'COPYHAND', 'moca_clock', 'MMSEASON', 'moca_letters', 
                                          'MMBALL', 'faq9', 'MMFLOOR', 'MMDRAW', 'MMMONTH', 'PXGENAPP', 'MMWATCH', 'CLOCKCIRC', 'faq8', 'MMHOSPIT', 'moca_naming', 
                                          'PXEXTREM', 'CLOCKNUM', 'PXMUSCUL', 'faq1', 'MMTREEDL', 'CLOCKTIME', 'PXABDOM', 'MMFLAG', 'COPYCIRC', 'MMAREA', 'faq3', 
                                          'moca_digits', 'CLOCKHAND', 'MMREAD', 'MMYEAR', 'MMREPEAT', 'moca_visuo_exec','MMHAND', 'MMBALLDL', 'PXCHEST', 'MMDATE', 
                                          'MMFLAGDL',  'MMSTATE','moca_repeat', 'MMFOLD', 'MMPENCIL', 'MMDAY',  'faq2', 'PXHEART', 'CLOCKSYM', 'faq4', 'moca_serial7', 
                                          'faq7', 'MMCITY', 'PXHEADEY', 'COPYTIME', 'PXPERIPH', 'PXSKIN', 'moca_fluency', 'moca_similarities', 'PXNECK', 'MMWRITE',
                                          'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
                                          'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg'])

data = pd.read_csv('../data/interim/data_adni.csv')
X = data[features]
y = data['CDGLOBAL']
y.replace({2:0,1:1},inplace=True)

In [2]:
X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)     

# RF Classifier

In [4]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 3.46 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.796000,0.871354,0.832908,0.833677,0.871354,0.796000,0.832908,0.833677,0.871354,0.796000,0.832908,0.833677
std,0.027415,0.050159,0.019366,0.019813,0.050159,0.027415,0.019366,0.019813,0.050159,0.027415,0.019366,0.019813
min,0.740000,0.781250,0.790816,0.790833,0.781250,0.740000,0.790816,0.790833,0.781250,0.740000,0.790816,0.790833
25%,0.777500,0.833333,0.816327,0.816771,0.833333,0.777500,0.816327,0.816771,0.833333,0.777500,0.816327,0.816771
50%,0.800000,0.880208,0.836735,0.836979,0.880208,0.800000,0.836735,0.836979,0.880208,0.800000,0.836735,0.836979
75%,0.812500,0.911458,0.848214,0.849792,0.911458,0.812500,0.848214,0.849792,0.911458,0.812500,0.848214,0.849792
max,0.840000,0.937500,0.857143,0.858125,0.937500,0.840000,0.857143,0.858125,0.937500,0.840000,0.857143,0.858125


## Random Search

In [5]:
from sklearn.model_selection import RandomizedSearchCV

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [6]:
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   51.2s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  6.2min finished


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(max_features=0.5,
                                                    random_state=555),
                   n_iter=400, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['sqrt', 'log2', 0.3,
                                                         0.5, 0.7, 0.9],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                        

In [7]:
rf_search_rand.best_params_

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 0.3,
 'max_depth': 100,
 'bootstrap': True}

In [8]:
best = rf_search_rand.best_estimator_
df_rand = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_rand.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.801111,0.932870,0.865646,0.866991,0.932870,0.801111,0.865646,0.866991,0.932870,0.801111,0.865646,0.866991
std,0.069362,0.035749,0.021343,0.020455,0.035749,0.069362,0.021343,0.020455,0.035749,0.069362,0.021343,0.020455
min,0.710000,0.885417,0.836735,0.839375,0.885417,0.710000,0.836735,0.839375,0.885417,0.710000,0.836735,0.839375
25%,0.710000,0.906250,0.841837,0.844583,0.906250,0.710000,0.841837,0.844583,0.906250,0.710000,0.841837,0.844583
50%,0.830000,0.927083,0.872449,0.872708,0.927083,0.830000,0.872449,0.872708,0.927083,0.830000,0.872449,0.872708
75%,0.860000,0.968750,0.877551,0.878542,0.968750,0.860000,0.877551,0.878542,0.968750,0.860000,0.877551,0.878542
max,0.860000,0.979167,0.892857,0.893958,0.979167,0.860000,0.892857,0.893958,0.979167,0.860000,0.892857,0.893958


## Grid Search

In [9]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_estimators': [100, 200, 300],
               'max_features': [.2, .3, .4],
               'max_depth': [90, 100, 110],
               'min_samples_split': [4, 5, 6],
               'min_samples_leaf': [3, 4, 5],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 682 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-1)]: Done 1112 tasks      | elapsed:   58.5s
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed:  1.3min finished


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(max_features=0.5,
                                              random_state=555),
             n_jobs=-1,
             param_grid={'bootstrap': [False, True],
                         'max_depth': [90, 100, 110],
                         'max_features': [0.2, 0.3, 0.4],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [4, 5, 6],
                         'n_estimators': [100, 200, 300]},
             verbose=2)

In [10]:
rf_search_grid.best_params_

{'bootstrap': True,
 'max_depth': 90,
 'max_features': 0.3,
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 200}

In [11]:
best = rf_search_grid.best_estimator_
df_grid = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_grid.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.801111,0.932870,0.865646,0.866991,0.932870,0.801111,0.865646,0.866991,0.932870,0.801111,0.865646,0.866991
std,0.069362,0.035749,0.021343,0.020455,0.035749,0.069362,0.021343,0.020455,0.035749,0.069362,0.021343,0.020455
min,0.710000,0.885417,0.836735,0.839375,0.885417,0.710000,0.836735,0.839375,0.885417,0.710000,0.836735,0.839375
25%,0.710000,0.906250,0.841837,0.844583,0.906250,0.710000,0.841837,0.844583,0.906250,0.710000,0.841837,0.844583
50%,0.830000,0.927083,0.872449,0.872708,0.927083,0.830000,0.872449,0.872708,0.927083,0.830000,0.872449,0.872708
75%,0.860000,0.968750,0.877551,0.878542,0.968750,0.860000,0.877551,0.878542,0.968750,0.860000,0.877551,0.878542
max,0.860000,0.979167,0.892857,0.893958,0.979167,0.860000,0.892857,0.893958,0.979167,0.860000,0.892857,0.893958


In [12]:
rf = RandomForestClassifier(n_estimators=200, max_features= .3, max_depth = 90, min_samples_split = 4, min_samples_leaf = 4, bootstrap=True, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 5.88 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.776000,0.895833,0.834694,0.835917,0.895833,0.776000,0.834694,0.835917,0.895833,0.776000,0.834694,0.835917
std,0.029982,0.043543,0.020899,0.021132,0.043543,0.029982,0.020899,0.021132,0.043543,0.029982,0.020899,0.021132
min,0.730000,0.843750,0.801020,0.801875,0.843750,0.730000,0.801020,0.801875,0.843750,0.730000,0.801020,0.801875
25%,0.755000,0.854167,0.821429,0.822083,0.854167,0.755000,0.821429,0.822083,0.854167,0.755000,0.821429,0.822083
50%,0.790000,0.895833,0.831633,0.832188,0.895833,0.790000,0.831633,0.832188,0.895833,0.790000,0.831633,0.832188
75%,0.800000,0.937500,0.841837,0.843854,0.937500,0.800000,0.841837,0.843854,0.937500,0.800000,0.841837,0.843854
max,0.810000,0.958333,0.882653,0.884167,0.958333,0.810000,0.882653,0.884167,0.958333,0.810000,0.882653,0.884167


In [14]:
# All samples

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(10)),repeats=list(int(x)*42+42 for x in range(10)), initial_split_seed=33433, initial_split_ratio=.25)
df.describe()

Wall time: 36.6 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.827350,0.914687,0.870128,0.871019,0.914687,0.827350,0.870128,0.871019,0.914687,0.827350,0.870128,0.871019
std,0.026624,0.016198,0.018057,0.017929,0.016198,0.026624,0.018057,0.017929,0.016198,0.026624,0.018057,0.017929
min,0.755000,0.869792,0.834184,0.835833,0.869792,0.755000,0.834184,0.835833,0.869792,0.755000,0.834184,0.835833
25%,0.815000,0.911458,0.857143,0.857812,0.911458,0.815000,0.857143,0.857812,0.911458,0.815000,0.857143,0.857812
50%,0.822500,0.916667,0.869898,0.870885,0.916667,0.822500,0.869898,0.870885,0.916667,0.822500,0.869898,0.870885
75%,0.850000,0.927083,0.887755,0.888281,0.927083,0.850000,0.887755,0.888281,0.927083,0.850000,0.887755,0.888281
max,0.875000,0.937500,0.905612,0.906250,0.937500,0.875000,0.905612,0.906250,0.937500,0.875000,0.905612,0.906250


# Outputs

In [15]:
### Save outputs ###
q = 'CDR5_Boruta17_2'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)